In [54]:
# import packages
from tensorflow.keras.layers import BatchNormalization, ZeroPadding1D, Conv1D, AveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dense,  Flatten, Input, add
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

#define resnet class
class ResNet:
  @staticmethod
  #create one residual block
  def residual_module(data, K, stride, chanDim=-1, reduce=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    shortcut = data
    
    bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
    act1 = Activation("relu")(bn1)
    conv1 = Conv1D(int(K*0.25), 1, use_bias=False, kernel_regularizer=l2(reg))(act1)

    bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
    act2 = Activation("relu")(bn2)
    conv2 = Conv1D(int(K * 0.25), 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act2)
    
    bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
    act3 = Activation("relu")(bn3)
    conv3 = Conv1D(K, 1, use_bias=False, kernel_regularizer=l2(reg))(act3)

    if reduce:
      shortcut = Conv1D(K, 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

    x = add([conv3, shortcut])

    return x

  @staticmethod
  #build a model out of resblocks
  def build(inputShape, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
    chanDim = -1
    inputs = Input(shape=inputShape)

    #initial batchnorm and convolutional layer
    x = BatchNormalization(axis=chanDim, epsilon = bnEps, momentum=bnMom)(inputs)
    x = Conv1D(filters[0], 5, use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
    
    x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
    x = Activation("relu")(x)
    x = ZeroPadding1D(1)(x)
    x = MaxPooling1D(3, strides=2)(x)

    for i in range(0, len(stages)):
      if i == 0:
        stride = 1
      else:
        stride = 2
      
      x = ResNet.residual_module(data=x, K=filters[i + 1], stride=stride, reduce=True, bnEps=bnEps, bnMom=bnMom)

      for j in range(0, stages[i] - 1):
        x = ResNet.residual_module(data=x, K=filters[i + 1], stride=1, bnEps=bnEps, bnMom=bnMom)
      
      x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
      x = Activation("relu")(x)
      x = AveragePooling1D(8)(x)

      x = Flatten()(x)
      x = Dense(classes, kernel_regularizer=l2(reg))(x)
      x = Activation("softmax")(x)

      model = Model(inputs, x, name="resnet")

      return model

  @staticmethod
  def pretrain(save_directory, features, labels, batch_size=32, epochs=10, stages=(6,), filters=(64, 100), resLayers=6, convLayers=4, verbose=2):
    X = features
    y = labels

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


    #add depth channel
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    #input shape is everything except for the number of samples 
    #number of classes is the number of unique items in y
    in_shape = X_train.shape[1:]
    n_classes = len(np.unique(y_train))

    #build model
    model = ResNet.build(inputShape=in_shape, classes=n_classes, stages=stages, filters=filters)

    #compile, train model
    model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs, verbose=verbose)

    #test accuracy
    accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    print("(pretraining) Testing accuracy from within X_reference.npy and y_reference.npy:", accuracy[1])

    # Save the weights
    model.save(save_directory)

    #return model
    return model

  @staticmethod
  def finetune(save_directory, features, labels, batch_size=10, epochs=10, verbose=2):
    X = features
    y = labels

    #create 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    
    model = tf.keras.models.load_model(save_directory)
  
    model.compile(optimizer=Adam(1e-5),  # Very low learning rate
    loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)
    loss, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    print("(finetuning) Testing accuracy from within X_fine and y_fine: ", acc)
  
    return model
  



In [55]:
#some hyperparameters
batch_size = 32
epochs = 10
verbose = 2
stages = (6,)
filters = (64, 100)
resLayers = 6
convLayers = 4
#save_directory is the path where the model will be saved after pretraining 
#and the path where the finetuning model will read from
save_directory = '/content/drive/My Drive/ML Group/Individual Work/Denzel/Saved Models/base.h5' 


#load data (specific to google drive)
X_test = np.load('/content/drive/My Drive/ML Group/Datasets/data/X_test.npy')
y_test = np.load('/content/drive/My Drive/ML Group/Datasets/data/y_test.npy')

X = np.load('/content/drive/My Drive/ML Group/Datasets/data/X_reference.npy')
y = np.load('/content/drive/My Drive/ML Group/Datasets/data/y_reference.npy')

X_fine = np.load('/content/drive/My Drive/ML Group/Datasets/data/X_finetune.npy')
y_fine = np.load('/content/drive/My Drive/ML Group/Datasets/data/y_finetune.npy')

pretrained_model = ResNet.pretrain(save_directory=save_directory, features=X, labels=y, 
                                   batch_size=batch_size, epochs=epochs, stages=stages, filters=filters, resLayers=resLayers, convLayers=convLayers)

pre_loss, pre_acc = pretrained_model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
print("Testing loss, accuracy after pretraining from X_test and y_test:", pre_loss, pre_acc)


Epoch 1/10
1500/1500 - 33s - loss: 0.5984 - accuracy: 0.8396
Epoch 2/10
1500/1500 - 33s - loss: 0.3442 - accuracy: 0.9125
Epoch 3/10
1500/1500 - 33s - loss: 0.3017 - accuracy: 0.9229
Epoch 4/10
1500/1500 - 33s - loss: 0.2756 - accuracy: 0.9295
Epoch 5/10
1500/1500 - 33s - loss: 0.2565 - accuracy: 0.9353
Epoch 6/10
1500/1500 - 33s - loss: 0.2475 - accuracy: 0.9387
Epoch 7/10
1500/1500 - 33s - loss: 0.2306 - accuracy: 0.9431
Epoch 8/10
1500/1500 - 33s - loss: 0.2252 - accuracy: 0.9448
Epoch 9/10
1500/1500 - 33s - loss: 0.2133 - accuracy: 0.9494
Epoch 10/10
1500/1500 - 33s - loss: 0.2040 - accuracy: 0.9518
375/375 - 3s - loss: 0.3397 - accuracy: 0.9125
(pretraining) Testing accuracy from within X_reference.npy and y_reference.npy: 0.9125000238418579
94/94 - 1s - loss: 3.7474 - accuracy: 0.5357
Testing loss, accuracy after pretraining from X_test and y_test: 3.747364044189453 0.5356666445732117


In [56]:

finetuned_model = ResNet.finetune(save_directory=save_directory, features=X_fine, labels=y_fine, epochs=65)

loss, acc = finetuned_model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
print("Pretrained accuracy: ", pre_acc)
print("Fintuned accuracy: ", acc)





Epoch 1/65
285/285 - 3s - loss: 3.5809 - accuracy: 0.5554
Epoch 2/65
285/285 - 3s - loss: 2.8981 - accuracy: 0.5881
Epoch 3/65
285/285 - 3s - loss: 2.3896 - accuracy: 0.6018
Epoch 4/65
285/285 - 3s - loss: 1.9868 - accuracy: 0.6242
Epoch 5/65
285/285 - 3s - loss: 1.7657 - accuracy: 0.6260
Epoch 6/65
285/285 - 3s - loss: 1.5502 - accuracy: 0.6519
Epoch 7/65
285/285 - 3s - loss: 1.4260 - accuracy: 0.6604
Epoch 8/65
285/285 - 3s - loss: 1.3331 - accuracy: 0.6691
Epoch 9/65
285/285 - 3s - loss: 1.2808 - accuracy: 0.6716
Epoch 10/65
285/285 - 3s - loss: 1.1958 - accuracy: 0.6860
Epoch 11/65
285/285 - 3s - loss: 1.1305 - accuracy: 0.6989
Epoch 12/65
285/285 - 3s - loss: 1.1106 - accuracy: 0.7018
Epoch 13/65
285/285 - 3s - loss: 1.0465 - accuracy: 0.7109
Epoch 14/65
285/285 - 3s - loss: 1.0036 - accuracy: 0.7225
Epoch 15/65
285/285 - 3s - loss: 0.9698 - accuracy: 0.7344
Epoch 16/65
285/285 - 3s - loss: 0.9452 - accuracy: 0.7365
Epoch 17/65
285/285 - 3s - loss: 0.9165 - accuracy: 0.7382
Epoch 